In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial columns.
application_df = application_df.drop("EIN", 1)
application_df = application_df.drop("NAME", 1)
application_df = application_df.drop("AFFILIATION", 1)
application_df.head()

/Users/bwheels/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/Users/bwheels/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/bwheels/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Look at APPLICATION_TYPE value counts for binning
app_type_count = application_df.APPLICATION_TYPE.value_counts()
app_type_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Determine which values to replace if counts are less than ...?
replace_application = list(app_type_count[app_type_count<800].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [6]:
# Determine which values to replace if counts are less than 1500
replace_class = list(classification_count[classification_count<1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

['APPLICATION_TYPE',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [8]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

/Users/bwheels/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

/Users/bwheels/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,CLASSIFICATION_C1000,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop(columns=["IS_SUCCESSFUL"]).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 90
hidden_nodes_layer2 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                3150      
                                                                 
 dense_1 (Dense)             (None, 40)                3640      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 6,831
Trainable params: 6,831
Non-trainable params: 0
_________________________________________________________________


2022-08-05 10:18:58.178173: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Create a callback that saves the model's weights every 5 epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
802/804 [============================>.] - ETA: 0s - loss: 0.6281 - accuracy: 0.6439
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6282 - accuracy: 0.6439
Epoch 2/100
800/804 [============================>.] - ETA: 0s - loss: 0.6169 - accuracy: 0.6507
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6169 - accuracy: 0.6505
Epoch 3/100
793/804 [============================>.] - ETA: 0s - loss: 0.6154 - accuracy: 0.6544
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6159 - accuracy: 0.6539
Epoch 4/100
777/804 [===========================>..] - ETA: 0s - loss: 0.6127 - accuracy: 0.6537
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6130 - accuracy: 0.6534
Epoch 5/100
804/804 [=======================

784/804 [============================>.] - ETA: 0s - loss: 0.6056 - accuracy: 0.6595
Epoch 35: saving model to checkpoints/weights.35.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6057 - accuracy: 0.6593
Epoch 36/100
800/804 [============================>.] - ETA: 0s - loss: 0.6051 - accuracy: 0.6596
Epoch 36: saving model to checkpoints/weights.36.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6052 - accuracy: 0.6597
Epoch 37/100
798/804 [============================>.] - ETA: 0s - loss: 0.6052 - accuracy: 0.6595
Epoch 37: saving model to checkpoints/weights.37.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6054 - accuracy: 0.6598
Epoch 38/100
801/804 [============================>.] - ETA: 0s - loss: 0.6055 - accuracy: 0.6584
Epoch 38: saving model to checkpoints/weights.38.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6054 - accuracy: 0.6588
Epoch 39/100
800/804 [===========================

776/804 [===========================>..] - ETA: 0s - loss: 0.6027 - accuracy: 0.6624
Epoch 69: saving model to checkpoints/weights.69.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6034 - accuracy: 0.6607
Epoch 70/100
783/804 [============================>.] - ETA: 0s - loss: 0.6036 - accuracy: 0.6586
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6035 - accuracy: 0.6592
Epoch 71/100
778/804 [============================>.] - ETA: 0s - loss: 0.6031 - accuracy: 0.6613
Epoch 71: saving model to checkpoints/weights.71.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6037 - accuracy: 0.6604
Epoch 72/100
798/804 [============================>.] - ETA: 0s - loss: 0.6034 - accuracy: 0.6593
Epoch 72: saving model to checkpoints/weights.72.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6033 - accuracy: 0.6595
Epoch 73/100
804/804 [===========================

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6155 - accuracy: 0.6591 - 481ms/epoch - 2ms/step
Loss: 0.6154897809028625, Accuracy: 0.6591253876686096


In [17]:
# Train the model 2.0
fit_model2 = nn.fit(X_train_scaled,y_train,epochs=50,callbacks=[cp_callback])

Epoch 1/50
800/804 [============================>.] - ETA: 0s - loss: 0.6023 - accuracy: 0.6610
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6025 - accuracy: 0.6607
Epoch 2/50
791/804 [============================>.] - ETA: 0s - loss: 0.6020 - accuracy: 0.6602
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 2s 3ms/step - loss: 0.6023 - accuracy: 0.6600
Epoch 3/50
779/804 [============================>.] - ETA: 0s - loss: 0.6022 - accuracy: 0.6609
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6021 - accuracy: 0.6607
Epoch 4/50
800/804 [============================>.] - ETA: 0s - loss: 0.6017 - accuracy: 0.6611
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6021 - accuracy: 0.6605
Epoch 5/50
798/804 [============================

795/804 [============================>.] - ETA: 0s - loss: 0.6022 - accuracy: 0.6607
Epoch 35: saving model to checkpoints/weights.35.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6021 - accuracy: 0.6609
Epoch 36/50
804/804 [==============================] - ETA: 0s - loss: 0.6025 - accuracy: 0.6604
Epoch 36: saving model to checkpoints/weights.36.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6025 - accuracy: 0.6604
Epoch 37/50
794/804 [============================>.] - ETA: 0s - loss: 0.6017 - accuracy: 0.6611
Epoch 37: saving model to checkpoints/weights.37.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6016 - accuracy: 0.6607
Epoch 38/50
794/804 [============================>.] - ETA: 0s - loss: 0.6014 - accuracy: 0.6609
Epoch 38: saving model to checkpoints/weights.38.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.6014 - accuracy: 0.6613
Epoch 39/50
780/804 [============================>.]

In [18]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6239 - accuracy: 0.6590 - 312ms/epoch - 1ms/step
Loss: 0.623924970626831, Accuracy: 0.6590087413787842


In [19]:
## model 3
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features3 = len(X_train[0])
hidden_nodes_layer13 = 80
hidden_nodes_layer23 = 30

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer13, input_dim=number_input_features3, activation="relu")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer23, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                2800      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,261
Trainable params: 5,261
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Create a callback that saves the model's weights every 5 epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

In [22]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=75,callbacks=[cp_callback])

Epoch 1/75
784/804 [============================>.] - ETA: 0s - loss: 0.6253 - accuracy: 0.6447
Epoch 1: saving model to checkpoints/weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6253 - accuracy: 0.6447
Epoch 2/75
804/804 [==============================] - ETA: 0s - loss: 0.6172 - accuracy: 0.6511
Epoch 2: saving model to checkpoints/weights.02.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6172 - accuracy: 0.6511
Epoch 3/75
804/804 [==============================] - ETA: 0s - loss: 0.6155 - accuracy: 0.6530
Epoch 3: saving model to checkpoints/weights.03.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6155 - accuracy: 0.6530
Epoch 4/75
789/804 [============================>.] - ETA: 0s - loss: 0.6131 - accuracy: 0.6541
Epoch 4: saving model to checkpoints/weights.04.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6130 - accuracy: 0.6544
Epoch 5/75
794/804 [============================

800/804 [============================>.] - ETA: 0s - loss: 0.6056 - accuracy: 0.6596
Epoch 35: saving model to checkpoints/weights.35.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6058 - accuracy: 0.6595
Epoch 36/75
782/804 [============================>.] - ETA: 0s - loss: 0.6059 - accuracy: 0.6603
Epoch 36: saving model to checkpoints/weights.36.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6059 - accuracy: 0.6603
Epoch 37/75
786/804 [============================>.] - ETA: 0s - loss: 0.6059 - accuracy: 0.6587
Epoch 37: saving model to checkpoints/weights.37.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6058 - accuracy: 0.6590
Epoch 38/75
787/804 [============================>.] - ETA: 0s - loss: 0.6058 - accuracy: 0.6591
Epoch 38: saving model to checkpoints/weights.38.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6056 - accuracy: 0.6600
Epoch 39/75
786/804 [============================>.]

792/804 [============================>.] - ETA: 0s - loss: 0.6036 - accuracy: 0.6596
Epoch 69: saving model to checkpoints/weights.69.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6035 - accuracy: 0.6600
Epoch 70/75
795/804 [============================>.] - ETA: 0s - loss: 0.6034 - accuracy: 0.6597
Epoch 70: saving model to checkpoints/weights.70.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6036 - accuracy: 0.6595
Epoch 71/75
796/804 [============================>.] - ETA: 0s - loss: 0.6040 - accuracy: 0.6605
Epoch 71: saving model to checkpoints/weights.71.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6039 - accuracy: 0.6603
Epoch 72/75
795/804 [============================>.] - ETA: 0s - loss: 0.6032 - accuracy: 0.6596
Epoch 72: saving model to checkpoints/weights.72.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6034 - accuracy: 0.6596
Epoch 73/75
798/804 [============================>.]

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6144 - accuracy: 0.6592 - 406ms/epoch - 2ms/step
Loss: 0.614386260509491, Accuracy: 0.6592419743537903
